In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime 
import pickle

from IPython.display import display
pd.options.display.max_columns = None

In [2]:
data_dir = "/home/julie/koupon-media/notebooks/julie_notebooks/python_practice/vision_zero/data/crash_data/"
data_files = os.listdir(data_dir)
data_files = [file for file in data_files if "txdot" in file]

In [3]:
data_files

['txdot_cris_crashes_hgac_201804_201806.csv',
 'txdot_cris_crashes_hgac_201807_201808.csv',
 'txdot_cris_crashes_hgac_201809.csv',
 'txdot_cris_crashes_hgac_201810_201901.csv']

In [4]:
#Find columns that are different among files
col0 = list(pd.read_csv(data_dir+data_files[0]).columns)
col1 = list(pd.read_csv(data_dir+data_files[1]).columns)
col2 = list(pd.read_csv(data_dir+data_files[2]).columns)
col3 = list(pd.read_csv(data_dir+data_files[3]).columns)

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col1)))))

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col2)))))

print(np.sort(list(set(list(col0))
 .symmetric_difference(list(col3)))))

print(np.sort(list(set(list(col1))
 .symmetric_difference(list(col2)))))

print(np.sort(list(set(list(col1))
 .symmetric_difference(list(col3)))))

print(np.sort(list(set(list(col2))
 .symmetric_difference(list(col3)))))


/data/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,78,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/data/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,78,100,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['file']
['file' 'month' 'year']
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount'
 'file' 'month' 'year']
['month' 'year']
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount'
 'month' 'year']
['CrashIncapacitatingInjuryCount' 'CrashNonSuspectedSeriousInjuryCount'
 'CrashNonincapacitatingInjuryCount' 'CrashSuspectedSeriousInjuryCount']


/data/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (26,43,78,100,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
agg_df = pd.read_csv(data_dir+data_files[0])

for file in data_files[1::]:
    df = pd.read_csv(data_dir+file)
    if "CrashNonSuspectedSeriousInjuryCount" in list(df.columns):
        df = df.rename(index = str, 
                       columns = {"CrashNonSuspectedSeriousInjuryCount":"CrashNonincapacitatingInjuryCount"})
    if "CrashSuspectedSeriousInjuryCount" in list(df.columns):
        df = df.rename(index = str, 
                       columns = {"CrashSuspectedSeriousInjuryCount":"CrashIncapacitatingInjuryCount"})
    agg_df = pd.concat([agg_df, df])

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [77]:
#Limit the dataframe and filter
keep_cols = ['CrashID',
'County', #filter for Harris, Montgomery, Fort Bend
'IntersectionRelated', # filter for intersection or intersection related
'SpeedLimit',
'RoadbedWidth',
'CrashSeverity',
'NumberofLanes',
'NumberofEnteringRoads',
'TrafficControlType',
'n_bike',
'n_cars',
'n_peds',
'n_train',
'RoadwayType',
'MedianWidth']

limit_df = agg_df.loc[:,keep_cols]

limit_df = limit_df.loc[(limit_df['County'].str.contains('Fort Bend')) |
                       (limit_df['County'].str.contains('Harris')) |
                       (limit_df['County'].str.contains('Montgomery'))]

limit_df = limit_df.loc[((limit_df['IntersectionRelated'].str.contains('Intersection')) |
                       (limit_df['IntersectionRelated'].str.contains('Intersection Related'))) &
                       ~(limit_df['IntersectionRelated'].str.contains('Non Intersection'))]

#Data Cleaning
#SpeedLimit - turn all negatives into NaN
#Round all numbers with %5 != 0
limit_df.loc[limit_df['SpeedLimit']<0, 'SpeedLimit'] = np.nan
limit_df.loc[limit_df['SpeedLimit']%5 != 0, 'SpeedLimit'] = round(limit_df.loc[limit_df['SpeedLimit']%5 != 0, 'SpeedLimit'],-1)

#RoadbedWidth - all No Data and nan to NaN
limit_df.loc[limit_df['RoadbedWidth']=='No Data', 'RoadbedWidth'] = np.nan
limit_df.loc[limit_df['RoadbedWidth']=='nan', 'RoadbedWidth'] = np.nan
limit_df['RoadbedWidth'] = limit_df['RoadbedWidth'].astype(float)

#Clean Severity
limit_df['CrashSeverity'] = limit_df['CrashSeverity'].str.upper()
limit_df.loc[limit_df['CrashSeverity']=='N - NOT INJURED', 'CrashSeverity'] = "NOT INJURED"
limit_df.loc[limit_df['CrashSeverity']=='C - POSSIBLE INJURY', 'CrashSeverity'] = "POSSIBLE INJURY"
limit_df.loc[limit_df['CrashSeverity']=='99 - UNKNOWN', 'CrashSeverity'] = "UNKNOWN"
limit_df.loc[limit_df['CrashSeverity']=='B - NON-INCAPACITATING INJURY', 'CrashSeverity'] = "NON-INCAPACITATING INJURY"
limit_df.loc[limit_df['CrashSeverity']=='A - SUSPECTED SERIOUS INJURY', 'CrashSeverity'] = "SUSPECTED SERIOUS INJURY"
limit_df.loc[limit_df['CrashSeverity']=='K - KILLED', 'CrashSeverity'] = "KILLED"

#Create a binary for CrashSeverity
limit_df['CrashSeverity_Binary'] = "NON-SEVERE"
limit_df.loc[(limit_df['CrashSeverity']=='SUSPECTED SERIOUS INJURY') |
             (limit_df['CrashSeverity']=='KILLED'), 'CrashSeverity_Binary'] = "SEVERE"

#NumberofLanes - all No Data and nan to NaN
limit_df.loc[limit_df['NumberofLanes']=='No Data', 'NumberofLanes'] = np.nan
limit_df.loc[limit_df['NumberofLanes']=='nan', 'NumberofLanes'] = np.nan
limit_df['NumberofLanes'] = limit_df['NumberofLanes'].astype(float)

#RoadwayType - all No Data and nan to NaN
limit_df.loc[limit_df['RoadwayType']=='No Data', 'RoadwayType'] = np.nan
limit_df.loc[limit_df['RoadwayType']=='nan', 'RoadwayType'] = np.nan

#MedianWidth - all No Data and nan to NaN
limit_df.loc[limit_df['MedianWidth']=='No Data', 'MedianWidth'] = np.nan
limit_df.loc[limit_df['MedianWidth']=='nan', 'MedianWidth'] = np.nan
limit_df['MedianWidth'] = limit_df['MedianWidth'].astype(float)

limit_df = limit_df.reset_index(drop = True)


In [81]:
limit_df.to_csv('limited_clean_txdot_dataset.csv')

In [82]:
limit_df.describe()

,CrashID,SpeedLimit,RoadbedWidth,NumberofLanes,n_bike,n_cars,n_peds,n_train,MedianWidth
count,3.801900e+04,36995.000000,11839.000000,11839.000000,38019.000000,38019.000000,38019.000000,38019.000000,11839.000000
mean,1.660534e+07,38.639816,90.782667,5.495650,0.007680,2.031931,0.012231,0.000789,16.857167
std,1.611234e+05,7.275604,42.871983,2.327537,0.088201,0.419902,0.120203,0.028080,29.111897
min,1.632950e+07,5.000000,20.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.646482e+07,35.000000,56.000000,4.000000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,1.659617e+07,35.000000,86.000000,6.000000,0.000000,2.000000,0.000000,0.000000,10.000000
75%,1.674481e+07,45.000000,120.000000,8.000000,0.000000,2.000000,0.000000,0.000000,20.000000
max,1.693274e+07,80.000000,282.000000,12.000000,2.000000,9.000000,4.000000,1.000000,280.000000
